# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,barranca,-10.7500,-77.7667,292.07,86,100,2.59,PE,1701667238
1,1,iqaluit,63.7506,-68.5145,260.00,78,100,6.17,CA,1701667171
2,2,constantia,44.1833,28.6500,274.65,61,100,5.88,RO,1701667238
3,3,edinburgh of the seven seas,-37.0676,-12.3116,287.77,86,1,5.67,SH,1701667239
4,4,new norfolk,-42.7826,147.0587,296.32,28,100,1.81,AU,1701667239


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
gmaps.configure(api_key=geoapify_key)

# Display the map
map_plot = city_data_df.hvplot.points('Lng',
                                    'Lat',
                                    geo=True,
                                    size='Humidity',
                                    scale=1,
                                    color='City',
                                    tiles='OSM',
                                    )

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_range = (city_data_df["Max Temp"] >= 250) & (city_data_df["Max Temp"] <= 350)
humidity_range = (city_data_df["Humidity"] >= 20) & (city_data_df["Humidity"] <= 50) 
wind_threshold = city_data_df["Wind Speed"] <= 15
cloudiness_threshold = city_data_df["Cloudiness"] <= 30

ideal_df = city_data_df[temp_range & humidity_range & wind_threshold & cloudiness_threshold].copy()

# Drop any rows with null values
ideal_df.dropna(inplace=True)


# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,ojinaga,29.5667,-104.4167,283.68,43,0,3.09,MX,1701667243
19,19,invercargill,-46.4000,168.3500,285.11,49,10,3.94,NZ,1701667243
37,37,sarakhs,36.5449,61.1577,290.56,24,0,3.56,IR,1701667247
45,45,dehloran,32.6941,47.2679,291.74,29,0,0.73,IR,1701667249
57,57,dongola,19.1667,30.4833,293.07,34,5,3.88,SD,1701667251


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
15,ojinaga,MX,29.5667,-104.4167,43,
19,invercargill,NZ,-46.4000,168.3500,49,
37,sarakhs,IR,36.5449,61.1577,24,
45,dehloran,IR,32.6941,47.2679,29,
57,dongola,SD,19.1667,30.4833,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories': "accommodation.hotel",
          'apiKey': geoapify_key,
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ojinaga - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
sarakhs - nearest hotel: No hotel found
dehloran - nearest hotel: No hotel found
dongola - nearest hotel: No hotel found
ma'rib - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
tafresh - nearest hotel: No hotel found
san angelo - nearest hotel: No hotel found
mutare - nearest hotel: No hotel found
yoloten - nearest hotel: No hotel found
linghai - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
pasni - nearest hotel: No hotel found
barmer - nearest hotel: No hotel found
hasaki - nearest hotel: No hotel found
zvishavane - nearest hotel: No hotel found
hotan - nearest hotel: No hotel found
nasir - nearest hotel: No hotel found
hafar al-batin - nearest hotel: No hotel found
mora - nearest hotel: No hotel found
calingasta - nearest hotel: No hotel found
medina - nearest hotel: No hotel found
reggane - nearest hotel: No hotel found
las vegas -

,City,Country,Lat,Lng,Humidity,Hotel Name
15,ojinaga,MX,29.5667,-104.4167,43,No hotel found
19,invercargill,NZ,-46.4000,168.3500,49,No hotel found
37,sarakhs,IR,36.5449,61.1577,24,No hotel found
45,dehloran,IR,32.6941,47.2679,29,No hotel found
57,dongola,SD,19.1667,30.4833,34,No hotel found
66,ma'rib,YE,15.4591,45.3253,38,No hotel found
95,hami,CN,42.8000,93.4500,28,No hotel found
96,tafresh,IR,34.6920,50.0130,23,No hotel found
99,san angelo,US,31.4638,-100.4370,44,No hotel found
106,mutare,ZW,-18.9707,32.6709,45,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotels_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    tiles = "OSM",
    hover_cols=list(hotel_df.columns)
    
)

# Display the map
hotels_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)